## Imports
---

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import time
import numpy as np
np.random.seed(42)

import torch
torch.manual_seed(42)

from torch_geometric.data import Data
from torch_geometric.nn import GATConv, BatchNorm, Linear
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.model_selection import ParameterGrid

from src.utils.dataset import get_full_transactions_dataset 

2025-03-06 11:29:22.979 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-03-06 11:29:22.989 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-03-06 11:29:22.989 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
print(f"Using Torch version {torch.__version__}")
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}") 
cuda_id = torch.cuda.current_device()
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Using Torch version 2.5.1+cu124
Is CUDA supported by this system? True
CUDA version: 12.4
Name of current CUDA device:NVIDIA GeForce RTX 2060


## Data set load
---

In [3]:
df = get_full_transactions_dataset()

2025-03-06 11:29:23.015 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-03-06 11:29:23.016 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:29:23.404 
  command:

    streamlit run C:\Users\ferna\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-06 11:29:23.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:29:23.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:29:23.911 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:29:23.911 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 11:29:31.479 Thread 'Ma

In [4]:
fraud_transactions_df = df[df["is_laundering"] == 1]
non_fraud_transactions_df = df[df["is_laundering"] == 0].sample(int(3e6))
df = pd.concat([fraud_transactions_df, non_fraud_transactions_df])

## Data preparation
---

### Preprocessing

In [5]:
label_encoder_columns = ["receiving_currency", "payment_currency", "payment_format"]
label_encoder = LabelEncoder()
for column in label_encoder_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [6]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df['timestamp'] = df['timestamp'].apply(lambda x: x.value)

In [7]:
df = df.sort_values(by="sender")

In [8]:
scaler = StandardScaler()
df[['amount_received', 'amount_paid', 'timestamp']] = scaler.fit_transform(df[['amount_received', 'amount_paid', 'timestamp']])

In [9]:
df.head()

,timestamp,sender,receiver,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering
2570346,0.088897,10057_803A115E0,29467_803E020C0,-0.005278,13,-0.004535,13,3,0
824238,-1.177713,10057_803A115E0,29467_803E020C0,-0.005378,13,-0.004658,13,4,0
3918768,0.979771,10057_803A115E0,29467_803E020C0,-0.005278,13,-0.004535,13,3,0
1541410,-0.883571,10057_803A115E0,29467_803E020C0,-0.005278,13,-0.004535,13,3,0
3437164,0.659839,10057_803A115E0,29467_803E020C0,-0.005378,13,-0.004658,13,4,0


### Node features

In [10]:
all_account_ids = set(df['sender']).union(set(df['receiver']))
nodes_df = pd.DataFrame({'account': list(all_account_ids)})
nodes_df = nodes_df.sort_values(by="account").reset_index(drop=True)
laundering_df = df[df["is_laundering"] == 1]
laundering_accounts = set(laundering_df['sender']).union(set(laundering_df['receiver']))
nodes_df["is_laundering"] = nodes_df.account.apply(lambda account_id: 1 if account_id in laundering_accounts else 0)
nodes_df = nodes_df.sort_values(by="account")

In [11]:
nodes_df["transactions_sent"] = nodes_df['account'].map(df.groupby('sender').size()).fillna(0)
nodes_df['transactions_received'] = nodes_df['account'].map(df.groupby('receiver').size()).fillna(0)
nodes_df['unique_currencies_sent'] = nodes_df['account'].map(df.groupby('sender')['payment_currency'].nunique()).fillna(0)
nodes_df['unique_currencies_received'] = nodes_df['account'].map(df.groupby('receiver')['payment_currency'].nunique()).fillna(0)

In [12]:
currencies = set(set(df['payment_currency']).union(set(df['receiving_currency'])))
for currency in currencies:
    nodes_df[f'average_paid_{currency}'] = nodes_df['account'].map(
        df[df['payment_currency'] == currency].groupby('sender')['amount_paid'].mean()
    ).fillna(0)
    
    nodes_df[f'total_received_{currency}'] = nodes_df['account'].map(
        df[df['receiving_currency'] == currency].groupby('receiver')['amount_received'].mean()
    ).fillna(0)

In [13]:
node_labels = torch.from_numpy(nodes_df["is_laundering"].values).to(torch.float)
nodes_df = nodes_df.drop(["account", "is_laundering"], axis=1)
node_features = torch.from_numpy(nodes_df.values).to(torch.float)

### Edge features

In [14]:
account_to_index = {acc: idx for idx, acc in enumerate(all_account_ids)}

In [15]:
edges_df = df.copy()

In [16]:
edges_df['sender'] = edges_df['sender'].map(account_to_index)
edges_df['receiver'] = edges_df['receiver'].map(account_to_index)

In [17]:
edges_df.head()

,timestamp,sender,receiver,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering
2570346,0.088897,192863,1082,-0.005278,13,-0.004535,13,3,0
824238,-1.177713,192863,1082,-0.005378,13,-0.004658,13,4,0
3918768,0.979771,192863,1082,-0.005278,13,-0.004535,13,3,0
1541410,-0.883571,192863,1082,-0.005278,13,-0.004535,13,3,0
3437164,0.659839,192863,1082,-0.005378,13,-0.004658,13,4,0


In [18]:
edge_index = torch.stack([torch.from_numpy(edges_df['sender'].values), torch.from_numpy(edges_df['receiver'].values)], dim=0)
edge_attr = torch.from_numpy(edges_df.drop(columns=["sender", "receiver", "is_laundering"]).values).to(torch.float)

### Graph data

In [19]:
graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=node_labels)

In [20]:
graph_data

Data(x=[456288, 34], edge_index=[2, 3005177], edge_attr=[3005177, 6], y=[456288])

### Undersampling
---

torch.bincount(graph_data.y.to(torch.long))

def remove_non_fraudulent_nodes(data, fraud_label=1, removal_percentage=0.5):
    non_fraud_nodes = torch.where(data.y != fraud_label)[0]
    num_to_remove = int(len(non_fraud_nodes) * removal_percentage)
    
    nodes_to_remove = np.random.choice(non_fraud_nodes.numpy(), num_to_remove, replace=False)
    nodes_to_remove = torch.tensor(nodes_to_remove, dtype=torch.long)
    
    mask = torch.ones(data.num_nodes, dtype=torch.bool)
    mask[nodes_to_remove] = False  
    remaining_nodes = torch.nonzero(mask, as_tuple=True)[0]  

    node_map = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(remaining_nodes)}
    mask_edges = mask[data.edge_index[0]] & mask[data.edge_index[1]]
    new_edge_index = data.edge_index[:, mask_edges]

    new_edge_index = torch.tensor([[node_map[idx.item()] for idx in row] for row in new_edge_index], dtype=torch.long)

    if data.edge_attr is not None:
        new_edge_attr = data.edge_attr[mask_edges]
    else:
        new_edge_attr = None

    new_data = data.__class__(
        x=data.x[remaining_nodes],  
        edge_index=new_edge_index,  
        edge_attr=new_edge_attr,  
        y=data.y[remaining_nodes]
    )
    
    return new_data

removal_percentage = 0.25
graph_data = remove_non_fraudulent_nodes(graph_data, fraud_label=1, removal_percentage=removal_percentage)

torch.bincount(graph_data.y.to(torch.long))

## Training
---

### Model definition

In [26]:
class GATModel(torch.nn.Module):
    def __init__(self, in_feats, hidden_dim, out_feats, heads):
        super().__init__()
        self.conv1 = GATConv(in_feats, hidden_dim, heads, dropout=0.2)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, concat=False, dropout=0.2)
        self.conv3 = GATConv(hidden_dim, int(hidden_dim/2), heads=1, concat=False, dropout=0.2)

        self.bn1 = BatchNorm(hidden_dim * heads)
        self.bn2 = BatchNorm(hidden_dim)
        self.bn3 = BatchNorm(int(hidden_dim/2))

        self.lin = Linear(int(hidden_dim/2), out_feats)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr):
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.bn1(F.leaky_relu(self.conv1(x, edge_index, edge_attr)))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.bn2(F.leaky_relu(self.conv2(x, edge_index, edge_attr)))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.bn3(F.leaky_relu(self.conv3(x, edge_index, edge_attr)))

        x = self.lin(x)
        x = self.sigmoid(x)

        return x

### Grid search

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
param_grid = {
    "lr": [1e-4],
    "batch_size": [1024],
    "num_neighbors": [[64,64], [256,256]],
    "hidden_dim": [16, 32, 64],
    "heads": [16, 32]
}

In [29]:
num_combinations = len(list(ParameterGrid(param_grid)))

In [30]:
num_combinations

12

In [31]:
train_test_split = T.RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0)
graph_data = train_test_split(graph_data).to(device)

In [32]:
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=graph_data.y.cpu().numpy())
pos_weight = torch.tensor([class_weights[1]], dtype=torch.float).to(device)

In [33]:
def create_loader(graph_data, mask, params):
    return NeighborLoader(
        graph_data,
        num_neighbors=params["num_neighbors"],
        batch_size=params["batch_size"],
        input_nodes=mask,
    )

In [34]:
best_params = None
best_auc = 0
param_combination = 1

In [ ]:
for params in ParameterGrid(param_grid):
    print(f"\nTesting parameters ({param_combination}/{num_combinations}): {params}")
    param_combination += 1
    print(f"Best AUC until now: {best_auc:.2f}")
    
    train_loader = create_loader(graph_data, graph_data.train_mask, params)
    test_loader = create_loader(graph_data, graph_data.val_mask, params)

    model = GATModel(
        in_feats=graph_data.num_features,
        hidden_dim=params["hidden_dim"],
        out_feats=1,
        heads=params["heads"]
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    num_epochs = 100
    for epoch in range(1, num_epochs + 1):
        start_time = time.time()
        model.train()
        running_loss = 0.0 
        
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.edge_attr)
            loss = criterion(out, batch.y.unsqueeze(1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        epoch_duration = time.time() - start_time

        if epoch % 10 == 0 or epoch == num_epochs:
            model.eval()
            y_true, y_pred_probs = [], []
            
            with torch.no_grad():
                for batch in test_loader:
                    out = model(batch.x, batch.edge_index, batch.edge_attr)
                    y_true.extend(batch.y.cpu().numpy())
                    y_pred_probs.extend(out.cpu().numpy())

            y_pred_probs = np.array(y_pred_probs).flatten()
            auc_score = roc_auc_score(y_true, y_pred_probs)

            if auc_score > best_auc:
                best_params = params
                best_auc = auc_score
            print(f"Epoch 
                  {epoch}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | AUC: {auc_score:.4f} | Time: {epoch_duration:.2f}s")


Testing parameters (1/12): {'batch_size': 1024, 'heads': 16, 'hidden_dim': 16, 'lr': 0.0001, 'num_neighbors': [64, 64]}
Best AUC until now: 0.00
Epoch 10/100 | Train Loss: 1.0585 | AUC: 0.6156 | Time: 9.96s
Epoch 20/100 | Train Loss: 1.0275 | AUC: 0.6249 | Time: 11.77s
Epoch 30/100 | Train Loss: 1.0241 | AUC: 0.6597 | Time: 10.89s
Epoch 40/100 | Train Loss: 1.0239 | AUC: 0.6589 | Time: 10.75s
Epoch 50/100 | Train Loss: 1.0233 | AUC: 0.6588 | Time: 10.81s
Epoch 60/100 | Train Loss: 1.0233 | AUC: 0.6561 | Time: 11.00s
Epoch 70/100 | Train Loss: 1.0231 | AUC: 0.6573 | Time: 11.07s
Epoch 80/100 | Train Loss: 1.0231 | AUC: 0.6596 | Time: 10.40s
Epoch 90/100 | Train Loss: 1.0224 | AUC: 0.6577 | Time: 9.13s
Epoch 100/100 | Train Loss: 1.0222 | AUC: 0.6599 | Time: 9.38s

Testing parameters (2/12): {'batch_size': 1024, 'heads': 16, 'hidden_dim': 16, 'lr': 0.0001, 'num_neighbors': [256, 256]}
Best AUC until now: 0.66
Epoch 10/100 | Train Loss: 1.0612 | AUC: 0.6715 | Time: 9.15s
Epoch 20/100 | T

{'batch_size': 4096, 'heads': 16, 'hidden_dim': 16, 'lr': 0.0001, 'num_neighbors': [64, 64]}

### Training best model

In [ ]:
params = {"batch_size": 512, "num_neighbors":[256, 256]}
heads=32
hidden_dim=32
lr=0.0001

In [ ]:
train_loader = create_loader(graph_data, graph_data.train_mask, params)
test_loader = create_loader(graph_data, graph_data.val_mask, params)

model = GATModel(
    in_feats=graph_data.num_features,
    hidden_dim=hidden_dim,
    out_feats=1,
    heads=heads
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    start_time = time.time()
    model.train()
    running_loss = 0.0 
    
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = criterion(out, batch.y.unsqueeze(1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    epoch_duration = time.time() - start_time

    if epoch % 10 == 0 or epoch == num_epochs:
        model.eval()
        y_true, y_pred_probs = [], []
        
        with torch.no_grad():
            for batch in test_loader:
                out = model(batch.x, batch.edge_index, batch.edge_attr)
                y_true.extend(batch.y.cpu().numpy())
                y_pred_probs.extend(out.cpu().numpy())

        y_pred_probs = np.array(y_pred_probs).flatten()
        auc_score = roc_auc_score(y_true, y_pred_probs)
        print(f"Epoch 
                {epoch}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | AUC: {auc_score:.4f} | Time: {epoch_duration:.2f}s")

### Evaluation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_curve, auc

def plot_roc_curve(y_true, y_pred_probs):
    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='grey', linestyle='--')  # Linha aleatória
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos (FPR)')
    plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
    plt.title('Curva ROC')
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

In [ ]:
model.eval()
y_true, y_pred_probs = [], []

with torch.no_grad():
    for batch in test_loader:
        out = model(batch.x, batch.edge_index, batch.edge_attr)
        y_true.extend(batch.y.cpu().numpy())
        y_pred_probs.extend(out.cpu().numpy())

y_pred_probs = np.array(y_pred_probs).flatten()
#y_pred = (y_pred_probs >= 0.1).astype(int)  # Converter para 0/1

In [ ]:
plot_roc_curve(y_true, y_pred_probs)